### This jupyter notebook is to match face of a given photo with all the faces from photos folder 

In [1]:
# importing libraries
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torchvision import datasets
from torch.utils.data import DataLoader
from PIL import Image

In [2]:

mtcnn = MTCNN(image_size=240, margin=0, min_face_size=20) # initializing mtcnn for face detection
resnet = InceptionResnetV1(pretrained='vggface2').eval() # initializing resnet for face img to embeding conversion

dataset=datasets.ImageFolder('photos') # photos folder path 
idx_to_class = {i:c for c,i in dataset.class_to_idx.items()} # accessing names of peoples from folder names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

face_list = [] # list of cropped faces from photos folder
name_list = [] # list of names corrospoing to cropped photos
embedding_list = [] # list of embeding matrix after conversion from cropped faces to embedding matrix using resnet

for img, idx in loader:
    face, prob = mtcnn(img, return_prob=True) 
    if face is not None and prob>0.90: # if face detected and porbability > 90%
        emb = resnet(face.unsqueeze(0)) # passing cropped face into resnet model to get embedding matrix
        embedding_list.append(emb.detach()) # resulten embedding matrix is stored in a list
        name_list.append(idx_to_class[idx]) # names are stored in a list
        


### Saving data into data.pt file

In [3]:
data = [embedding_list, name_list]
torch.save(data, 'data.pt') # saving data.pt file

### Matching face id of the given photo with available data from data.pt file

In [4]:

def face_match(img_path, data_path): # img_path= location of photo, data_path= location of data.pt 
    # getting embedding matrix of the given img
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True) # returns cropped face and probability
    emb = resnet(face.unsqueeze(0)).detach() # detech is to make required gradient false
    
    saved_data = torch.load('data.pt') # loading data.pt file
    embedding_list = saved_data[0] # getting embedding data
    name_list = saved_data[1] # getting list of names
    dist_list = [] # list of matched distances, minimum distance is used to identify the person
    
    for idx, emb_db in enumerate(embedding_list):
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
        
    idx_min = dist_list.index(min(dist_list))
    return (name_list[idx_min], min(dist_list))


result = face_match('test2.JPG', 'data.pt')

print('Face matched with: ',result[0], 'With distance: ',result[1])

Face matched with:  abubakar_saddique With distance:  0.5929115414619446


In [11]:
def face_match(img_path, data_path, threshold=0.6):
    img = Image.open(img_path)
    face, prob = mtcnn(img, return_prob=True)
    
    # Return a message if face detection fails or probability is low
    if face is None or prob < 0.90:
        return ("No face detected", None)
    
    emb = resnet(face.unsqueeze(0)).detach()
    
    saved_data = torch.load(data_path)
    embedding_list = saved_data[0]
    name_list = saved_data[1]
    
    dist_list = []
    
    for emb_db in embedding_list:
        dist = torch.dist(emb, emb_db).item()
        dist_list.append(dist)
    
    min_dist = min(dist_list)
    
    if min_dist <= threshold:
        idx_min = dist_list.index(min_dist)
        return (name_list[idx_min], min_dist)
    else:
        return ("No match found", min_dist)

result = face_match('WIN_20240511_14_17_55_Pro.jpg', 'data.pt', threshold=0.6)

print('Face matched with: ', result[0], 'With distance: ', result[1])

Face matched with:  abubakar_saddique With distance:  0.41614869236946106


## MTCNN x -> R-CNN

In [16]:
# If not installed yet, install torchvision
!pip install torchvision


Defaulting to user installation because normal site-packages is not writeable


In [17]:
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader
from torchvision import datasets
from PIL import Image
from facenet_pytorch import InceptionResnetV1

# Step 2: Initialize Faster R-CNN for Face Detection

In [18]:
# Initialize a Faster R-CNN model with a pre-trained backbone
faster_rcnn = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
faster_rcnn.eval()  # Set to evaluation mode

C:\Users\Abubakar\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Abubakar\AppData\Roaming\Python\Python39\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to C:\Users\Abubakar/.cache\torch\hub\checkpoints\fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|████████████████████████████████████████████████████████████████████████████████| 160M/160M [03:23<00:00, 821kB/s]


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

## Step 3: Define a Function to Extract Faces Using Faster R-CNN

In [23]:
def detect_faces(image):
    # Transform the image into a format expected by the model
    transform = transforms.Compose([transforms.ToTensor()])
    img_tensor = transform(image)

    # Run the image through Faster R-CNN to get the bounding boxes
    with torch.no_grad():
        detections = faster_rcnn([img_tensor])

    # Confidence threshold to determine valid detections
    confidence_threshold = 0.7
    valid_boxes = [box for box, score in zip(detections[0]['boxes'], detections[0]['scores']) if score >= confidence_threshold]

    if valid_boxes:
        # Select the box with the highest confidence
        box = valid_boxes[0]

        # Convert tensor coordinates to integers for cropping
        x1, y1, x2, y2 = [int(coord) for coord in box]

        # Extract the face region from the image
        cropped_face = image.crop((x1, y1, x2, y2))

        # Transform the cropped face to a PyTorch tensor
        cropped_face_tensor = transform(cropped_face)
        
        return cropped_face_tensor
    else:
        return None

## Step 4: Integrate Faster R-CNN with the Rest of the Code

In [27]:
# Initialize InceptionResnetV1 for embedding extraction
resnet = InceptionResnetV1(pretrained='vggface2').eval()

# Initialize dataset and DataLoader
dataset = datasets.ImageFolder('photos')  # Photos folder path
idx_to_class = {i: c for c, i in dataset.class_to_idx.items()}  # Class names

def collate_fn(x):
    return x[0]

loader = DataLoader(dataset, collate_fn=collate_fn)

# Initialize lists for face embeddings and names
face_list = []
name_list = []
embedding_list = []

for img, idx in loader:
    face = detect_faces(img)  # Use Faster R-CNN to extract the face
    # Determine if the tensor is valid based on its size or shape
    if face is not None and torch.numel(face) > 0:
        emb = resnet(face.unsqueeze(0))  # Create embedding
        embedding_list.append(emb.detach())
        name_list.append(idx_to_class[idx])  # Store the corresponding name

    
# Save embeddings and names
data = [embedding_list, name_list]
torch.save(data, 'data.pt')  # Save to a file

# Face matching function using Faster R-CNN
def face_match(img_path, data_path, threshold=0.6):
    img = Image.open(img_path)
    face = detect_faces(img)  # Use Faster R-CNN to extract the face
    
    # Check if face is None or has zero elements
    if face is None or torch.numel(face) == 0:
        return ("No face detected", None)

    # Transform face to a PyTorch tensor and extract embedding
    emb = resnet(face.unsqueeze(0)).detach()

    # Load the stored data for comparison
    saved_data = torch.load(data_path)  # Load saved data
    embedding_list = saved_data[0]
    name_list = saved_data[1]

    # Calculate distances between embeddings
    dist_list = []
    for emb_db in embedding_list:
        dist = torch.dist(emb, emb_db).item()  # Calculate distance
        dist_list.append(dist)

    # Find the minimum distance
    min_dist = min(dist_list)

    # Determine if it's a match based on the threshold
    if min_dist <= threshold:
        idx_min = dist_list.index(min_dist)
        return (name_list[idx_min], min_dist)
    else:
        return ("No match found", min_dist)


# Test the modified face match function
result = face_match('test3.JPG', 'data.pt', threshold=0.6)

print('Face matched with: ', result[0], 'With distance: ', result[1])

KeyboardInterrupt: 